# NIRISS commissioning kernel-phase data analysis

This notebook walks through the analysis of NIRISS CLEARP kernel-phase data starting from uncal files and finishing with companion detection limits. Installation instructions are given at the top.

## Step 1: install and import libraries

In addition to the JWST KPI3 pipeline, two extra libraries are required to run this notebook:
1. the JWST data reduction pipeline (https://github.com/spacetelescope/jwst),
2. Jens Kammerer's fouriever toolkit (https://github.com/kammerje/fouriever).

In the virtual environment where you installed the KPI3Pipeline, do the following:
1. Install the JWST data reduction pipeline
```
python -m pip install jwst
```
2. clone fouriever into a directory of your choice
```
python -m pip install git+https://github.com/kammerje/fouriever.git
```

In [1]:
import os

# Set up CRDS environment variables (required for JWST data reduction pipeline)
# Optinally uncomment if your environment variables are not set
# os.environ['CRDS_PATH'] = 'crds_cache'
# os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

# Import JWST data reduction pipeline
from jwst.pipeline import Detector1Pipeline, Image2Pipeline

# Import kernel-phase pipeline from XARA
from jwst_kpi.calwebb_kpi3 import KPI3Pipeline

# Import model fitting tools from fouriever
from fouriever import klcal, uvfit

## Step 2: run JWST stage 1 & 2 and KPI stage 3 pipelines

The ipc, the photom, and the resample step in the JWST stage 1 & 2 pipelines should be skipped for kernel-phase data, similar as for AMI data. The KPI stage 3 pipeline creates diagnostic plots for each pipeline step that are saved in the output directory of the pipeline. The KPI stage 3 pipeline saves the output into KPFITS files whose structure is outlined here: https://docs.google.com/document/d/1iBbcCYiq9J2PpLSr21-xB4AXP8X_6tSszxnHY1VDGXg/edit?usp=sharing.

The input data is the NIRISS commissioning data. It can be downloaded from MAST.
The `uncal` files are expected to be separated per target as shown below

In [ ]:
# Get the uncal files
idirs = ['kerphase_testdata/NIRISS/J062802.01-663738.0/',
         'kerphase_testdata/NIRISS/TYC-8906-1660-1/',
         'kerphase_testdata/NIRISS/CPD-66-562/',
         'kerphase_testdata/NIRISS/CPD-67-607/']
files = []
for idir in idirs:
    files += sorted([idir+f for f in os.listdir(idir) if f.endswith('_uncal.fits')])

In [2]:
# Run each file through the pipelines
show_plots = True
for file in files:

    output_dir = os.path.dirname(file) + os.sep

    # Stage 1
    result1 = Detector1Pipeline()
    result1.save_results = True
    result1.output_dir = output_dir
    result1.ipc.skip = True
    result1.run(file)

    # Stage 2
    result2 = Image2Pipeline()
    result2.save_results = True
    result2.output_dir = output_dir
    result2.photom.skip = True
    result2.resample.skip = True
    result2.run(file.replace('uncal', 'rateints'))

    # Stage 3
    result3 = KPI3Pipeline()
    result3.output_dir = None  # output directory; if 'None', uses same directory as input data
    result3.show_plots = show_plots
    # Bad pixel fixing step
    result3.fix_bad_pixels.skip = False  # skip step?
    result3.fix_bad_pixels.plot = True  # make and save plots?
    result3.fix_bad_pixels.bad_bits = ['DO_NOT_USE']  # DQ flags to be considered as bad pixels (see https://jwst-reffiles.stsci.edu/source/data_quality.html)
    result3.fix_bad_pixels.method = 'medfilt'  # method to fix bad pixels; 'medfilt' or 'KI'
    # Re-centering step
    result3.recenter_frames.skip = False
    result3.recenter_frames.plot = True
    result3.recenter_frames.method = 'FPNM'  # XARA re-centering method; 'BCEN', 'COGI', or 'FPNM'
    result3.recenter_frames.crop = True
    result3.recenter_frames.bmax = 6.  # in m: maximum baseline length for FPNM re-centering method
    # Windowing step
    result3.window_frames.skip = False
    result3.window_frames.plot = True
    result3.window_frames.wrad = None  # in pix: radius of super-Gaussian window mask; if 'None', makes automatic guess
    # Kernel-phase extraction step
    result3.extract_kerphase.skip = False
    result3.extract_kerphase.plot = True
    result3.extract_kerphase.bmax = None  # m; maximum baseline length for kernel-phase extraction; if 'None', uses entire pupil model
    # Empirical uncertainties step
    result3.empirical_uncertainties.skip = False
    result3.empirical_uncertainties.plot = True
    result3.empirical_uncertainties.get_emp_err = True  # estimate uncertainties empirically from standard deviation over individual frames?
    # Run pipeline
    result3.run(file.replace('uncal', 'calints'))

    show_plots = False

--> Running fix bad pixels step...
Found 123365 bad pixels (8.03%)
Done
--> Running extract kerphase step...
Rotating pupil model by -0.57 deg (counter-clockwise)
Attempting to load file pupil_model.txt
230 distinct baselines were identified
first 10 singular values for this array:
[14.89703 11.54589 10.39097  8.80026  7.44934  6.31752  6.24093  5.66995
  5.45849  5.11   ]
<xara.kpi.KPI object at 0x161e30490> KPI data structure
----------------------------------------
->  64 sub-apertures
-> 230 distinct baselines
-> 167 Ker-phases ( 72.6 % target phase)
->  63 Eig-phases (100.0 % wavefront phase)
----------------------------------------

KPI data successfully loaded
File provided is not a fits file
No KPO data included

First time for m2pix = 5.29: 
LDFT1: Computing new Fourier matrix...
Done!
Image shift = (5.03, 0.05)
Image shift = (5.04, -0.22)
Image shift = (5.02, 0.06)
Image shift = (5.02, 0.06)
Image shift = (5.03, 0.05)
Image shift = (5.02, 0.05)
Image shift = (5.03, 0.06)
Imag

First time for m2pix = 5.29: 
LDFT1: Computing new Fourier matrix...
Done!
Image shift = (9.55, 4.54)
Image shift = (9.49, 4.53)
Image shift = (9.48, 4.53)
Image shift = (9.55, 4.54)
Image shift = (9.55, 4.54)
Image shift = (9.55, 4.53)
Image shift = (9.55, 4.54)
Image shift = (9.54, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.55, 4.54)
Image shift = (9.55, 4.53)
Image shift = (9.49, 4.53)
Image shift = (9.55, 4.53)
Image shift = (9.48, 4.52)
Image shift = (9.55, 4.53)
Image shift = (9.55, 4.54)
Image shift = (9.54, 4.54)
Image shift = (9.49, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.49, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.49, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.55, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.49, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.49, 4.53)
Image shift = (9.55, 4.53)
Image shift = (9.55, 4.53)
Image shift = (9.54, 4.53)
Image shift = (9.55, 4.54)
Image s

Image shift = (4.89, -0.02)
Image shift = (4.88, -0.02)
Image shift = (4.89, -0.01)
Image shift = (4.89, -0.01)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.01)
Image shift = (4.90, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.01)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.88, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.88, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.01)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.03)
Image shift = (4.89, -0.03)
Image shift = (4.89, -0.01)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.01)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89, -0.02)
Image shift = (4.89,

Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.30, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.35, 4.47)
Image shift = (9.35, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.34, 4.45)
Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.36, 4.46)
Image shift = (9.30, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.35, 4.46)
Image shift = (9.34, 4.45)
Image shift = (9.27, 4.47)
Image shift = (9.36, 4.46)
I

Image shift = (5.03, 0.08)
Image shift = (5.03, 0.09)
Image shift = (5.02, 0.08)
Image shift = (5.03, 0.08)
Image shift = (5.02, 0.08)
Image shift = (5.03, 0.09)
Image shift = (5.02, 0.09)
Image shift = (5.01, 0.08)
Image shift = (5.03, 0.08)
Image shift = (5.03, 0.08)
Image shift = (5.02, 0.09)
Image shift = (5.03, 0.08)
Image shift = (5.03, 0.08)
Image shift = (5.02, 0.09)
Image shift = (5.02, 0.09)
Image shift = (5.02, 0.09)
Image shift = (5.03, 0.09)
Image shift = (5.02, 0.08)
Image shift = (5.02, 0.09)
Image shift = (5.02, 0.09)
Image shift = (5.03, 0.09)
Image shift = (5.01, 0.09)
Image shift = (5.03, 0.08)
Image shift = (5.03, 0.08)
Image shift = (5.02, 0.09)
Image shift = (5.03, 0.08)
Image shift = (5.03, 0.08)
Image shift = (5.05, 0.16)
Image shift = (5.02, 0.08)
Image shift = (5.03, 0.09)
Image shift = (5.02, 0.09)
Image shift = (5.03, 0.09)
Image shift = (5.02, 0.09)
Image shift = (5.02, 0.09)
Image shift = (5.02, 0.09)
Image shift = (5.03, 0.09)
Image shift = (5.03, 0.09)
I

Image shift = (9.54, 4.57)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.57)
Image shift = (9.54, 4.57)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.57)
Image shift = (9.55, 4.57)
Image shift = (9.55, 4.57)
Image shift = (9.54, 4.56)
Image shift = (9.54, 4.57)
Image shift = (9.55, 4.57)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.57)
Image shift = (9.55, 4.57)
Image shift = (9.54, 4.56)
Image shift = (9.55, 4.57)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.57)
Image shift = (9.54, 4.56)
Image shift = (9.54, 4.57)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.54, 4.57)
Image shift = (9.55, 4.57)
Image shift = (9.54, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.56)
Image shift = (9.55, 4.57)
I

Image shift = (4.89, -0.06)
Image shift = (4.85, -0.12)
Image shift = (4.89, -0.06)
Image shift = (4.88, -0.06)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.07)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.07)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.05)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.07)
Image shift = (4.90, -0.07)
Image shift = (4.89, -0.06)
Image shift = (4.84, -0.11)
Image shift = (4.89, -0.07)
Image shift = (4.89, -0.06)
Image shift = (4.90, -0.07)
Image shift = (4.88, -0.06)
Image shift = (4.89, -0.07)
Image shift = (4.91, -0.00)
Image shift = (4.89, -0.06)
Image shift = (4.90, -0.07)
Image shift = (4.88, -0.07)
Image shift = (4.89, -0.06)
Image shift = (4.88, -0.07)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.06)
Image shift = (4.89, -0.07)
Image shift = (4.89, -0.06)
Image shift = (4.89,

Image shift = (9.35, 4.41)
Image shift = (9.36, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.32, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.41)
Image shift = (9.36, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.36, 4.42)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.42)
Image shift = (9.36, 4.41)
Image shift = (9.36, 4.41)
Image shift = (9.35, 4.41)
Image shift = (9.35, 4.41)
I

## Step 3: determine contrast limits with fouriever

Fouriever can directly read the KPFITS files from the KPI stage 3 pipeline. The raw 1-sigma noise floor can be computed with the lincmap function in uvfit.

In [ ]:
fouriever_raw_prefix = os.path.join(os.path.dirname(os.path.dirname(idirs[0])), "raw")

In [3]:
# Get the KPFITS files of J062802.01-663738.0
fitsfiles = [f.replace('uncal', 'calints_emp_kpfits') for f in files[:2]]

# Load the KPFITS files into fouriever
data = uvfit.data(idir='',
                  fitsfiles=fitsfiles)

# Compute 1-sigma noise floor
fit = data.lincmap(
    cov=False,  # account for data covariance in fit?
    sep_range=(30., 600.),  # mas; separation range of grid
    step_size=10.,  # mas; step size of grid
    smear=None,  # use bandwidth smearing?
    ofile=fouriever_raw_prefix,  # output file name
    save_as_fits=True
)  # save output as FITS file?
data.detlim(
    sigma=3.0,  # confidence level of detection limits
    fit_sub=None,  # best fit from MCMC
    cov=False,  # account for data covariance in fit?
    sep_range=(30., 600.),  # mas; separation range of grid
    step_size=40.,  # mas; step size of grid
    smear=None,  # use bandwidth smearing?
    ofile=fouriever_raw_prefix,
)  # output file name

Opened NIRISS data
   1 observations
   230 Fourier phases
   167 kernel phases
   1 wavelengths
Opened NIRISS data
   1 observations
   230 Fourier phases
   167 kernel phases
   1 wavelengths
Selected instrument = NIRISS
   Use self.set_inst(inst) to change the selected instrument
Selected observables = ['kp']
   Use self.set_observables(observables) to change the selected observables
Data properties
   Smallest spatial scale = 92.5 mas
   Largest spatial scale = 924.9 mas
   Using data covariance = False
Computing grid
   Min. sep. = 100.0 mas
   Max. sep. = 1000.0 mas
   31112 non-empty grid cells
Computing linear contrast map (DO NOT TRUST UNCERTAINTIES)
   Cell 40401 of 40401
   Best fit companion flux = 2.523e-02 +/- 7.590e-07
   Best fit companion right ascension = -130.0 mas
   Best fit companion declination = -270.0 mas
   Best fit companion separation = 299.7 mas
   Best fit companion position angle = -154.3 deg
   Best fit red. chi2 = 4.315 (bin)
   Significance of companio

In kernel-phase imaging (and AMI), the kernel-phase of the science target is usually calibrated with the kernel-phase of a point-source reference target. Fouriever can perform a Karhunen-Loeve calibration using the calibrate function in klcal. After calibrating the data, the calibrated 1-sigma noise floor can be computed.

In [4]:
# Output directory where the calibrated KPFITS files are saved
odir = os.path.join(os.path.dirname(os.path.dirname(idirs[0])), "cal/")

# Get the KPFITS files of CPD-66-562
scifiles = [f.replace('uncal', 'calints_emp_kpfits') for f in files[4:6]]

# Get the KPFITS files of J062802.01-663738.0, TYC-8906-1660-1, and CPD-67-607
calfiles = [f.replace('uncal', 'calints_emp_kpfits') for f in [*files[:4], *files[6:]]]

Selected instrument = NIRISS
   Use self.set_inst(inst) to change the selected instrument
Selected observables = ['kp']
   Use self.set_observables(observables) to change the selected observables
Computing Karhunen-Loeve decomposition
   K_klip = 1
   kp: 6 data sets
   kp: projection matrix shape = (166, 167)
Computing Karhunen-Loeve projection
   File 2 of 2: kernel phase FITS file
Opened NIRISS data
   1 observations
   230 Fourier phases
   166 kernel phases
   1 wavelengths
Opened NIRISS data
   1 observations
   230 Fourier phases
   166 kernel phases
   1 wavelengths
Selected instrument = NIRISS
   Use self.set_inst(inst) to change the selected instrument
Selected observables = ['kp']
   Use self.set_observables(observables) to change the selected observables
Data properties
   Smallest spatial scale = 92.5 mas
   Largest spatial scale = 924.9 mas
   Using data covariance = False
Computing grid
   Min. sep. = 100.0 mas
   Max. sep. = 1000.0 mas
   31112 non-empty grid cells
Comp

In [ ]:
# Load the KPFITS files into fouriever
data = klcal.data(scidir='',
                  scifiles=scifiles,
                  caldir='',
                  calfiles=calfiles)

# Perform Karhunen-Loeve calibration
data.calibrate(odir=odir,
               K_klip=3)

In [ ]:
# Get the calibrated KPFITS files
fitsfiles = [f for f in os.listdir(odir) if f.endswith('emp_kpfits_klcal.fits')]
fouriever_cal_prefix = os.path.join(os.path.dirname(os.path.dirname(idirs[0])), "cal")

# Load the KPFITS files into fouriever
data = uvfit.data(idir=odir,
                  fitsfiles=fitsfiles)

# Compute 1-sigma noise floor
fit = data.lincmap(
    cov=False,  # account for data covariance in fit?
    sep_range=(30., 600.),  # mas; separation range of grid
    step_size=10.,  # mas; step size of grid
    smear=None,  # use bandwidth smearing?
    ofile=fouriever_cal_prefix,  # output file name
    save_as_fits=True
)
data.detlim(
    sigma=3.0,  # confidence level of detection limits
    fit_sub=None,  # best fit from MCMC
    cov=False,  # account for data covariance in fit?
    sep_range=(30., 600.),  # mas; separation range of grid
    step_size=40.,  # mas; step size of grid
    smear=None,  # use bandwidth smearing?
    ofile=fouriever_cal_prefix,
)

In [ ]:
fit = data.chi2map(
    model="bin",
    cov=False,
    sep_range=(30., 600.),
    step_size=40.,
    smear=None,
    ofile=fouriever_cal_prefix,
)

data.mcmc(fit=fit, temp=None, smear=None, ofile=fouriever_cal_prefix)